In [ ]:
#mount your drive to access the files from Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Output 
Mounted at /content/drive/


print("**************************************")
print("Libraries are imported successfully. ")
print("**************************************")
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI


In [ ]:
print("*************************************************")
print("Environment variables are created successfully. ")
print("*************************************************")

#Creating environment variables
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
os.environ["PINECONE_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', xxxxxxxxxxxxxxxxxxx')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')


In [ ]:
#llm is ready with minimal temperature.
llm = OpenAI(OpenAI=api_keys, temperature=0.1)


# initialize pinecone
import pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to the api key in the console
)

index_name = "abcdefg" # put the name of your pinecone index here

# provide the path of pdf files.
pdfreader = PdfReader(filepath)

Loop through the file pages and get the content.

from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

Print the content

raw_text


In [ ]:
# We need to split the text using Character Text Split

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)


#find the length of the overall text
len(texts)



In [ ]:
# Creating embeddings of objects from OpenAI
embeddings = OpenAIEmbeddings()

#conneceting embedding object and text object along with Pinecone index
print(“*************************”)
print(“Pinecone object is ready.”)
print(“*************************”)
vectordb = Pinecone.from_texts(texts,embeddings,index_name="demoindex")

Output
Pinecone objects are ready.

# Create a retriever object to query the vector database
retriever = vectordb.as_retriever(score_threshold = 0.3)

chain = load_qa_chain(OpenAI(), chain_type="stuff")

query = "Who is the author?"
docs = vectordb.similarity_search(query, k=3)
chain.run(input_documents=docs, question=query)


In [ ]:
docs = vectordb.similarity_search("What is age of the author")
chain.run(input_documents=docs, question=query)


In [ ]:
Combining Multiple Chains Using simple Sequential Chain

from langchain.chains import LLMChain
master_template=PromptTemplate(input_variables=['Technical requirements'],template="Please tell me about {Technical requirements}")
level_1_chain=LLMChain(llm=llm,prompt=master_template)

References_template=PromptTemplate(input_variables=['Further reading'],template="Suggest me some References and further reading {Further reading}")
level_2_chain=LLMChain(llm=llm,prompt=References_template)
from langchain.chains import SimpleSequentialChain
comb_chain=SimpleSequentialChain(chains=[level_1_chain,level_2_chain])
comb_chain.run("Deep Learning Containers")
